In [ ]:
# excel_ingestion.py

from pathlib import Path
import pandas as pd
import os

def parse_excel_file(filepath: Path) -> dict:
    xls = pd.ExcelFile(filepath)
    required_sheets = ["Journal", "Executions", "Statistics"]
    for sheet in required_sheets:
        if sheet not in xls.sheet_names:
            raise ValueError(f"El archivo {filepath.name} no contiene la hoja '{sheet}'")
    
    return {
        "source_file": filepath.name,
        "journal": pd.read_excel(xls, sheet_name="Journal"),
        "executions": pd.read_excel(xls, sheet_name="Executions"),
        "statistics": pd.read_excel(xls, sheet_name="Statistics")
    }


# process_trades.py
import pandas as pd

def build_trade_dataframe(journal_df, executions_df, source_file, capital_base=600):
    n_trades = len(journal_df)
    n_execs = len(executions_df)
    if n_execs < 2:
        raise ValueError("No hay suficientes ejecuciones.")

    max_trades = min(n_trades, n_execs // 2)
    trades_df = journal_df.iloc[:max_trades].copy()

    trades_df = trades_df.rename(columns={
        "Open time": "entry_time",
        "Close time": "exit_time",
        "Instrument": "asset",
        "Open price": "entry_price",
        "Close price": "exit_price",
        "Open volume": "position_size",
        "PnL": "PnL",
        "Profit (ticks)": "profit_ticks",
        "Account": "account"
    })

    trades_df["direction"] = trades_df["position_size"].apply(lambda x: "Buy" if x > 0 else "Sell")
    trades_df["order_id_entry"] = executions_df.iloc[::2]["Exchange ID"].values[:len(trades_df)]
    trades_df["order_id_exit"] = executions_df.iloc[1::2]["Exchange ID"].values[:len(trades_df)]

    trades_df["source_file"] = source_file
    trades_df["commission"] = executions_df["Commission"].values[:2*len(trades_df)].reshape(-1, 2).sum(axis=1)
    trades_df["PnL_net"] = trades_df["PnL"] - trades_df["commission"]

    trades_df["entry_time"] = pd.to_datetime(trades_df["entry_time"])
    trades_df["exit_time"] = pd.to_datetime(trades_df["exit_time"])
    trades_df["duration_minutes"] = (trades_df["exit_time"] - trades_df["entry_time"]).dt.total_seconds() / 60

    equity_start = capital_base + trades_df["PnL"].cumsum().iloc[0]
    trades_df["equity"] = equity_start + trades_df["PnL"].cumsum()

    return trades_df


# database_io.py
from sqlalchemy import create_engine

def save_to_postgres(trades_df, daily_stats, user="alex", db="trading"):
    engine = create_engine(f"postgresql+psycopg2://{user}@localhost:5432/{db}")
    trades_df.to_sql("trades", engine, if_exists="replace", index=False)
    daily_stats.to_sql("daily_summary", engine, if_exists="append", index=False)


# validators.py
def validate_required_columns(df, required):
    missing = set(required) - set(df.columns)
    if missing:
        raise ValueError(f"Faltan columnas requeridas: {missing}")




In [2]:
# =======================================
# config.py
# =======================================

CAPITAL_BASE_GLOBAL = 1000

# =======================================
# utils.py
# =======================================

import pandas as pd
from config import CAPITAL_BASE_GLOBAL

def calcular_capital_actual(df: pd.DataFrame) -> float:
    if df.empty:
        return CAPITAL_BASE_GLOBAL
    if "equity" in df.columns and df["equity"].iloc[-1] > 0:
        return df["equity"].iloc[-1]
    return CAPITAL_BASE_GLOBAL + df["PnL"].cumsum().iloc[-1] if 'PnL' in df.columns else CAPITAL_BASE_GLOBAL

# =======================================
# load_data.py
# =======================================

import os
from pathlib import Path
import pandas as pd

def load_existing_data(output_dir):
    hist_path = output_dir / "trades_hist.csv"
    sum_path = output_dir / "trading_summary.csv"

    if hist_path.exists():
        hist_df = pd.read_csv(hist_path)
        if "source_file" not in hist_df.columns:
            hist_df["source_file"] = ""
    else:
        hist_df = pd.DataFrame()

    if sum_path.exists():
        summary_df = pd.read_csv(sum_path)
        if "source_file" not in summary_df.columns:
            summary_df["source_file"] = ""
    else:
        summary_df = pd.DataFrame()

    return hist_df, summary_df, hist_path, sum_path

# =======================================
# excel_ingestion.py
# =======================================

import pandas as pd
from pathlib import Path

def parse_excel_file(filepath: Path):
    xls = pd.ExcelFile(filepath)
    required_sheets = ["Journal", "Executions", "Statistics"]
    for sheet in required_sheets:
        if sheet not in xls.sheet_names:
            raise ValueError(f"El archivo {filepath.name} no contiene la hoja '{sheet}'")
    return {
        "source_file": filepath.name,
        "journal": pd.read_excel(xls, sheet_name="Journal"),
        "executions": pd.read_excel(xls, sheet_name="Executions"),
        "statistics": pd.read_excel(xls, sheet_name="Statistics")
    }

# =======================================
# process_trades.py
# =======================================

import pandas as pd
import numpy as np
from sqlalchemy import create_engine

def build_trade_dataframe(journal_df, executions_df, source_file, capital_base=600):
    n_trades = len(journal_df)
    n_execs = len(executions_df)
    if n_execs < 2:
        raise ValueError("No hay suficientes ejecuciones.")

    max_trades = min(n_trades, n_execs // 2)
    trades_df = journal_df.iloc[:max_trades].copy()

    trades_df = trades_df.rename(columns={
        "Open time": "entry_time",
        "Close time": "exit_time",
        "Instrument": "asset",
        "Open price": "entry_price",
        "Close price": "exit_price",
        "Open volume": "position_size",
        "PnL": "PnL",
        "Profit (ticks)": "profit_ticks",
        "Account": "account"
    })

    trades_df["direction"] = trades_df["position_size"].apply(lambda x: "Buy" if x > 0 else "Sell")
    trades_df["order_id_entry"] = executions_df.iloc[::2]["Exchange ID"].values[:len(trades_df)]
    trades_df["order_id_exit"] = executions_df.iloc[1::2]["Exchange ID"].values[:len(trades_df)]
    trades_df["source_file"] = source_file

    raw_commissions = executions_df["Commission"].values[:2 * len(trades_df)]
    if len(raw_commissions) % 2 != 0:
        raise ValueError("Comisiones mal formateadas")
    trades_df["commission"] = raw_commissions.reshape(-1, 2).sum(axis=1)
    trades_df["PnL_net"] = trades_df["PnL"] - trades_df["commission"]

    trades_df["entry_time"] = pd.to_datetime(trades_df["entry_time"])
    trades_df["exit_time"] = pd.to_datetime(trades_df["exit_time"])
    trades_df["duration_minutes"] = (trades_df["exit_time"] - trades_df["entry_time"]).dt.total_seconds() / 60

    equity_start = capital_base + trades_df["PnL"].cumsum().iloc[0]
    trades_df["equity"] = equity_start + trades_df["PnL"].cumsum()

    return trades_df

def save_to_postgres(trades_df, daily_stats, user="alex", db="trading"):
    engine = create_engine(f"postgresql+psycopg2://{user}@localhost:5432/{db}")
    trades_df.to_sql("trades", engine, if_exists="replace", index=False)
    daily_stats.to_sql("daily_summary", engine, if_exists="append", index=False)

# =======================================
# main.py
# =======================================

from pathlib import Path
from load_data import load_existing_data
from bayesian_model import build_bayesian_params, bayesian_mc_simulation
from streak_analysis import analyze_streaks
from equity_simulation import plot_equity_simulation
from utils import calcular_capital_actual
from excel_ingestion import parse_excel_file
from process_trades import build_trade_dataframe, save_to_postgres
import matplotlib.pyplot as plt
import pandas as pd
import os

input_dir = Path("input")
output_dir = Path("output")
input_dir.mkdir(exist_ok=True)
output_dir.mkdir(exist_ok=True)

hist_df, summary_df, hist_path, sum_path = load_existing_data(output_dir)

excel_files = sorted(input_dir.glob("*.xlsx"), key=os.path.getmtime)

for archivo in excel_files:
    try:
        data = parse_excel_file(archivo)
        if data["source_file"] in hist_df.get("source_file", []).astype(str).values:
            continue  # ya procesado

        trades_df = build_trade_dataframe(data["journal"], data["executions"], data["source_file"])
        stats_df = data["statistics"].set_index("Name").T.reset_index(drop=True)
        stats_df["source_file"] = data["source_file"]

        hist_df = pd.concat([hist_df, trades_df], ignore_index=True)
        hist_df.drop_duplicates(subset=["order_id_entry", "order_id_exit"], inplace=True)
        hist_df.to_csv(hist_path, index=False)

        summary_df = pd.concat([summary_df, stats_df], ignore_index=True)
        summary_df.to_csv(sum_path, index=False)

        save_to_postgres(trades_df, stats_df)
        print(f"✅ Procesado: {archivo.name}")

    except Exception as e:
        print(f"❌ Error procesando {archivo.name}: {e}")

if hist_df.empty:
    print("⚠️ No hay datos para analizar.")
else:
    if 'PnL' not in hist_df.columns:
        print("❌ Faltan columnas necesarias en el historial: 'PnL'")
    else:
        params = build_bayesian_params(hist_df)
        risk_of_ruin = bayesian_mc_simulation(params)
        print(f"\n🔍 Riesgo de ruina estimado: {risk_of_ruin * 100:.2f}%\n")

        win_rate_emp = (hist_df['PnL'] > 0).mean()
        streak_df = analyze_streaks(win_rate_emp, len(hist_df))
        print(streak_df.head())

        plot_equity_simulation(
            win_rate=55,
            win_loss_ratio=2,
            risk_per_trade=1,
            n_trades=100,
            n_lines=10,
            trades_df=hist_df
        )

        print("✅ Análisis completo ejecutado.")


ModuleNotFoundError: No module named 'excel_ingestion'